In [1]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import Row, SQLContext, SparkSession, window
from pyspark import SparkConf
# Import data types
from pyspark.sql.types import *
conf = SparkConf()
conf.setMaster("local[6]")
conf.set("spark.network.timeout","800s")
conf.set("spark.driver.memory","100g")
conf.set("spark.memory.storageFraction","0.2")
conf.set("spark.driver.maxResultSize","0")
spark = SparkSession.builder.appName("pcap_analyzer").config(conf=conf).getOrCreate()

In [ ]:
#multipleline
pcapPath = "/home/antslab/data_hdd4t/pcap_process/2020_01_10/中華電信/snort.2020-01-10_dir/snort.log/*.json"
pcap_df_multiline = spark.read.option("multiline", "true").json(pcapPath)

In [ ]:
# 重新整理schema
# layer_df = pcap_df_multiline.select("_source.layers.`ip.src`")
# src_ip_df = layer_df.select("`ip.src`")

In [6]:
src_ip_df = pcap_df_multiline.select("_source.layers.`ip.src`")
src_ip_df.createOrReplaceTempView("src_ip")

## 計算當日不重複IP數

In [ ]:
distinctquery = (spark.sql("SELECT COUNT(DISTINCT `ip.src`) FROM src_ip"))
distinctresult = distinctquery.head()

In [ ]:
result

## 計算當日流量 Top 10 IP

In [ ]:
maxflowquery = (spark.sql("SELECT `ip.src`, COUNT(*) FROM src_ip GROUP BY `ip.src` ORDER BY COUNT(*) DESC LIMIT 10"))

In [ ]:
# 直接拿maxflowquery來畫圖
maxflowresult = maxflowquery.collect()

In [ ]:
maxflowresult

## 計算單一IP在特定time interval (e.g. second, minute, hour)下最多封包數量

In [7]:
from pyspark.sql import functions as f
from pyspark.sql.types import *

In [8]:
# df = pcap_df_multiline.select("_source.layers.`ip.src`","_source.layers.`frame.time`")
df = pcap_df_multiline.select(f.col("_source.layers.`ip.src`").getItem(0).alias("ip.src"),f.col("_source.layers.`frame.time`").getItem(0).alias("frame.time"))

In [10]:
df = df.withColumn("frame.time",f.to_timestamp(f.col("`frame.time`"),"MMM dd, yyyy HH:mm:ss.SSSSSSSSS z"))

In [11]:
df.printSchema()

root
 |-- ip.src: string (nullable = true)
 |-- frame.time: timestamp (nullable = true)



In [ ]:
hour_result = df.groupBy("`ip.src`" ,f.window("`frame.time`","1 hour")).count().orderBy('count', ascending=False)

In [ ]:
a = hour_result.collect()

In [ ]:
import pickle as pk
with open("/home/antslab/spark_data/intermediate/pcap/cht_0110_perhour.pk", "wb") as file:
    pk.dump(a, file)

In [12]:
ms_result = df.groupBy("`ip.src`" ,f.window("`frame.time`","1 milliseconds")).count().orderBy('window', ascending=True)

In [13]:
b = ms_result.collect()

In [ ]:
import pickle as pk
with open("/home/antslab/spark_data/intermediate/pcap/cht_0110_ms.pk", "wb") as file:
    pk.dump(b, file)

## 之前找外部來源IP top10結果 (cht 1/10)

In [ ]:
# import pickle as pk
# with open("cht_0110_top10.pk", 'wb') as file:
#     pk.dump(result, file)

In [ ]:
import pickle as pk
with open("/home/antslab/spark_data/intermediate/pcap/cht_0110_top10.pk", 'rb') as file:
    aresult = pk.load(file)